In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

In [36]:
a = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])

In [6]:
a.shape

(4, 3)

In [7]:
np.random.permutation(3)

array([1, 2, 0])

In [153]:
def batch_generator(X, y, shuffle=True, batch_size=2):
    """
    Гератор новых батчей для обучения
    X          - матрица объекты-признаки
    y_batch    - вектор ответов
    shuffle    - нужно ли случайно перемешивать выборку
    batch_size - размер батча ( 1 это SGD, > 1 mini-batch GD)
    Генерирует подвыборку для итерации спуска (X_batch, y_batch)
    """
    n_batches = X.shape[0] // batch_size
    for i in range(n_batches):
        mask = np.random.permutation(X.shape[0])[:batch_size]
        X_batch, y_batch = X[mask], y[mask]
        yield (X_batch, y_batch)

In [10]:
from sklearn.datasets import load_iris

In [18]:
X_train = load_iris()['data']
y_train = load_iris()['target']

In [45]:
X_train = a
y_train = np.array([1, 1, 1, 1])

In [89]:
w = np.array([-1,2,-3])

In [47]:
def sigmoid(X):
    """
    Вычисляем значение сигмоида.
    X - матрица объекты-признаки
    """
    return 1 / (1 + np.exp(-X))

In [99]:
X_train[:, 0]

array([ 1,  4,  7, 10])

In [100]:
X_train

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [101]:
a_i = sigmoid(X_train.dot(w))

In [55]:
y_train = np.array([0,1,2,3])

In [94]:
np.sum(y_train * np.log(a_i) + (1-y_train) * np.log(1-a_i))

-120.00248184459895

In [102]:
a_i

array([  2.47262316e-03,   6.14417460e-06,   1.52299795e-08,
         3.77513454e-11])

In [105]:
tmp = a_i-y_train

In [104]:
X_train

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [109]:
X_train

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [128]:
buf = tmp.dot(X_train)

In [134]:
tmp = X_train.dot(w)

SyntaxError: invalid syntax (<ipython-input-138-af96b098b2de>, line 1)

In [352]:
#%%pycodestyle

def sigmoid(X):
    """
    Вычисляем значение сигмоида.
    X - матрица объекты-признаки
    """
    return 1 / (1 + np.exp(-X))


from sklearn.base import BaseEstimator, ClassifierMixin

class MySGDClassifier(BaseEstimator, ClassifierMixin):
    """
    Предполагается, что в выборке всегда 2 класса
    """
    
    def __init__(self, batch_generator, C=1, alpha=0.01, max_epoch=10):
        """
        batch_generator -- функция генератор, которой будем создавать батчи
        C - коэф. регуляризации
        alpha - скорость спуска
        max_epoch - максимальное количество эпох
        """
        
        self.C = C
        self.alpha = alpha
        self.max_epoch = max_epoch
        self.batch_generator = batch_generator
        self.errors_log = {'iter' : [], 'loss' : []}
        
    def calc_loss(self, X_batch, y_batch):
        """
        Считаем функцию потерь по батчу 
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        """
        a = sigmoid(X_batch.dot(self.weights))
        R = np.sum(self.weights**2)/self.C
        N = X_batch.shape[0]
        return -(1/N)*np.sum(y_batch*np.log(a)+(1-y_batch)*np.log(1-a))+R
    
    def calc_loss_grad(self, X_batch, y_batch):
        """
        Считаем  градиент функции потерь по батчу (то что Вы вывели в задании 1)
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        """
        a = sigmoid(X_batch.dot(self.weights))
        N = X_batch.shape[0]
        return (1/N)*(a-y_batch).dot(X_batch)+(2/self.C)*self.weights
    
    def update_weights(self, new_grad):
        """
        Обновляем вектор весов
        new_grad - градиент по батчу
        """
        self.weights -= self.alpha*new_grad
    
    def fit(self, X, y):
        '''
        Обучение модели
        X - матрица объекты-признаки
        y - вектор ответов
        '''
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        self.weights = np.random.uniform(0, 1, X.shape[1])
        for n in range(0, self.max_epoch):
            new_epoch_generator = self.batch_generator(X, y)
            for batch_num, new_batch in enumerate(new_epoch_generator):
                X_batch = new_batch[0]
                y_batch = new_batch[1]
                batch_grad = self.calc_loss_grad(X_batch, y_batch)
                self.update_weights(batch_grad)
                # Подумайте в каком месте стоит посчитать ошибку для отладки модели
                # > После шага градиентного спуска посчитайте 
                # значение ошибки на объекте 
                batch_loss = self.calc_loss(X_batch, y_batch)
                self.errors_log['iter'].append(batch_num)
                self.errors_log['loss'].append(batch_loss)
                
        return self
        
    def predict(self, X):
        '''
        Предсказание класса
        X - матрица объекты-признаки
        '''
        
        # Желательно здесь использовать матричные операции между X и весами, например, numpy.dot
        return (self.predict_proba(X)>=0.5).astype(int)
    
    def predict_proba(self, X):
        '''
        Предсказание вероятности положительного класса
        X - матрица объекты-признаки
        '''
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        y_hat_proba = sigmoid(X.dot(self.weights))
        # Желательно здесь использовать матричные операции между X и весами, например, numpy.dot 
        return y_hat_proba



In [312]:
np.random.seed(0)

C1 = np.array([[0., -0.8], [1.5, 0.8]])
C2 = np.array([[1., -0.7], [2., 0.7]])
gauss1 = np.dot(np.random.randn(200, 2) + np.array([5, 3]), C1)
gauss2 = np.dot(np.random.randn(200, 2) + np.array([1.5, 0]), C2)
X = np.vstack([gauss1, gauss2])
y = np.r_[np.ones(200), np.zeros(200)]


In [353]:
ss = MySGDClassifier(batch_generator, C = 100)

In [307]:
X = np.vstack([gauss1, gauss2])
y = np.r_[np.ones(200), np.zeros(200)]

In [354]:
ss.fit(X, y)

MySGDClassifier(C=100, alpha=0.01,
        batch_generator=<function batch_generator at 0x113282378>,
        max_epoch=10)

In [257]:
X1 = np.hstack((np.ones((X.shape[0], 1)), X))

In [351]:
zz = np.array([True, False])
zz = zz.astype(int)
zz

array([1, 0])

In [272]:
a = sigmoid(X1.dot(weights))
N = X1.shape[0]
# (1/N)*X.dot(a-y)+(2/self.C)*weights

In [268]:
weights = np.random.uniform(0, 1, X1.shape[1])

In [270]:
X1.shape

(400, 3)

In [343]:
a = ss.predict_proba(X)>0.5

In [344]:
np.unique(a, return_counts=True)

(array([False,  True], dtype=bool), array([151, 249]))

In [355]:
ss.predict(X)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0,

In [ ]:
X.dot()

In [183]:
a = ss.predict_proba(X)>=0.5

In [184]:
np.unique(a, return_counts=True)

(array([False,  True], dtype=bool), array([ 48, 352]))

In [185]:
X_copy = np.hstack((np.ones((X.shape[0], 1)), X))

In [232]:
N = 3
a = np.random.rand(N,N)
b = np.zeros((N,N+1))
b[:,:-1] = a

In [248]:
a = np.ones((X.shape[0], X.shape[1]+1))

In [247]:
z = np.ones((X.shape[0], X.shape[1]+1))

(400, 2)

In [249]:
a[:, 1:] = X

In [251]:
def kappa(X):
    X = X[:1]

In [253]:
z = np.array([1,2,3,4])
kappa(z)
z

array([1, 2, 3, 4])